# Profiling

<a href="https://colab.research.google.com/github/Ziaeemehr/workshop_hpcpy/blob/main/notebooks/profiling/01_line_profiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
import numpy as np

In [2]:
def euclidean_broadcast(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]

    return (diff * diff).sum(axis=2)

In [4]:
def euclidean_trick(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]

    xy = x @ y.T

    return np.abs(x2 + y2 - 2. * xy)

In [6]:
nrows = 2000
ncols = 50

rng = np.random.default_rng()
x = 10. * rng.random((nrows, ncols))
y = 10. * rng.random((nrows, ncols))
print(np.allclose(euclidean_broadcast(x, y), euclidean_trick(x, y)))

True


### 1. `timeit`

In [7]:
%timeit euclidean_broadcast(x, y)
%timeit euclidean_trick(x, y)

947 ms ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
37.6 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 2. `line_profiler`

In [ ]:
# ! conda install conda-forge::line_profiler -y

In [8]:
%load_ext line_profiler

In [9]:
%lprun -f euclidean_broadcast euclidean_broadcast(x,x)

Timer unit: 1e-09 s

Total time: 0.947309 s
File: /tmp/ipykernel_1258576/3677175976.py
Function: euclidean_broadcast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def euclidean_broadcast(x, y):
     2                                               """Euclidean square distance matrix.
     3                                               
     4                                               Inputs:
     5                                               x: (N, m) numpy array
     6                                               y: (N, m) numpy array
     7                                               
     8                                               Ouput:
     9                                               (N, N) Euclidean square distance matrix:
    10                                               r_ij = (x_ij - y_ij)^2
    11                                               """
    12         1  438652966.0    

In [10]:
%lprun -f euclidean_trick euclidean_trick(x, x)

Timer unit: 1e-09 s

Total time: 0.0320159 s
File: /tmp/ipykernel_1258576/2952558958.py
Function: euclidean_trick at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def euclidean_trick(x, y):
     2                                               """Euclidean square distance matrix.
     3                                               
     4                                               Inputs:
     5                                               x: (N, m) numpy array
     6                                               y: (N, m) numpy array
     7                                               
     8                                               Ouput:
     9                                               (N, N) Euclidean square distance matrix:
    10                                               r_ij = (x_ij - y_ij)^2
    11                                               """
    12         1     617117.0 617117.0  

### 3. `cProfile`

In [11]:
%prun -r euclidean_trick(x, x)

         18 function calls in 0.036 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.035    0.035    0.035    0.035 2952558958.py:1(euclidean_trick)
        1    0.001    0.001    0.036    0.036 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.c_einsum}
        1    0.000    0.000    0.036    0.036 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 einsumfunc.py:1009(einsum)
       10    0.000    0.000    0.000    0.000 einsumfunc.py:1001(_einsum_dispatcher)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

`Memory profiling`

In [11]:
# ! conda install conda-forge::memory_profiler -y

In [12]:
%%bash 
python 02_memprofiler.py

Filename: /home/ziaee/git/workshops/hpcpy2024/notebooks/profiling/02_memprofiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     5     59.7 MiB     59.7 MiB           1   @profile
     6                                         def euclidean_trick(x, y):
     7                                             """Euclidean square distance matrix.
     8                                         
     9                                             Inputs:
    10                                             x: (N, m) numpy array
    11                                             y: (N, m) numpy array
    12                                         
    13                                             Ouput:
    14                                             (N, N) Euclidean square distance matrix:
    15                                             r_ij = (x_ij - y_ij)^2
    16                                             """
    17     59.7 MiB      0.0 MiB           1       x2 =

In [13]:
%%bash 
python 03_memprofiler.py

Filename: /home/ziaee/git/workshops/hpcpy2024/notebooks/profiling/03_memprofiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     5     59.6 MiB     59.6 MiB           1   @profile
     6                                         def euclidean_broadcast(x, y):
     7                                             """Euclidean square distance matrix.
     8                                         
     9                                             Inputs:
    10                                             x: (N, m) numpy array
    11                                             y: (N, m) numpy array
    12                                         
    13                                             Ouput:
    14                                             (N, N) Euclidean square distance matrix:
    15                                             r_ij = (x_ij - y_ij)^2
    16                                             """
    17   1585.5 MiB   1525.9 MiB           1       